In [63]:
import pandas as pd

df = pd.read_csv('100_Unique_QA_Dataset.csv')

df.head()

question      answer
0                   What is the capital of France?       Paris
1                  What is the capital of Germany?      Berlin
2               Who wrote 'To Kill a Mockingbird'?  Harper-Lee
3  What is the largest planet in our solar system?     Jupiter
4   What is the boiling point of water in Celsius?         100

In [64]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [65]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [66]:
# vocab
vocab = {'<UNK>':0}

In [67]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [68]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [69]:
len(vocab)

324

In [70]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [71]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [72]:
import torch
from torch.utils.data import Dataset, DataLoader

In [73]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [74]:
dataset = QADataset(df, vocab)

In [75]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [76]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([166])
tensor([[ 10,  96,   3, 104, 239]]) tensor([240])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([205])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([106])
tensor([[ 42,  86,  87, 241, 242,  19,  39, 243]]) tensor([244])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([321])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([273])
tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([65])
tensor([[  1,   2,   3,  37, 133,   5,  26]]) tensor([134])
tensor([[  1,   2,   3,   4,   5, 236, 237]]) tensor([238])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([58])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([36])
tensor([[  1,   2,   3,  92, 137,  19,   3,  45]]) tensor([185])
tensor([[ 10,  29, 130, 131]]) tensor([132])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]]) tensor([205])
tensor([[10, 96,  3, 97]]) tensor([98])
tensor([[  1,   2,   3, 141, 117,  83,   3

In [77]:
import torch.nn as nn

In [78]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [79]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [80]:
learning_rate = 0.001
epochs = 20

In [81]:
model = SimpleRNN(len(vocab))

In [82]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [83]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 523.743765
Epoch: 2, Loss: 458.663857
Epoch: 3, Loss: 381.366082
Epoch: 4, Loss: 316.583935
Epoch: 5, Loss: 263.613381
Epoch: 6, Loss: 214.377218
Epoch: 7, Loss: 170.555124
Epoch: 8, Loss: 132.431492
Epoch: 9, Loss: 101.580636
Epoch: 10, Loss: 77.406278
Epoch: 11, Loss: 59.737802
Epoch: 12, Loss: 46.832662
Epoch: 13, Loss: 37.391672
Epoch: 14, Loss: 30.122496
Epoch: 15, Loss: 24.743959
Epoch: 16, Loss: 20.663754
Epoch: 17, Loss: 17.417320
Epoch: 18, Loss: 14.886125
Epoch: 19, Loss: 12.775326
Epoch: 20, Loss: 11.076138


In [84]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [85]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [86]:
list(vocab.keys())[7]

'paris'